In [86]:
import os
import warnings
import pandas as pd
import requests
from bs4 import BeautifulSoup
import wget
import ssl
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

data_atual = datetime.now()
data_atual_f = data_atual.date().strftime('%d-%m-%Y')
data_ontem = data_atual - timedelta(days=1)
data_ontem = data_ontem.date().strftime('%d-%m-%Y')

ssl._create_default_https_context = ssl._create_unverified_context

url = f'https://extrator.sefaz.al.gov.br/DESPESAS/COMPARATIVO-DESPESAS/despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem}.xlsx'
arquivo = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem}.xlsx')
arquivo_csv = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem}.csv')
base_arquivo = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/SCRIPTS/Bases empenhado - Layne', f'Base Acompanhamento - atualizado {data_atual_f}.xlsx')

# Baixar o arquivo
wget.download(url, arquivo)
df = pd.read_excel(arquivo)

df_f = df[['DESCRICAO_UO', 'PT', 'PT_DESCRICAO', 'NATUREZA6', 'VALOR_EMPENHADO', 'MES']]
for i in ['DESCRICAO_UO', 'PT', 'PT_DESCRICAO', 'NATUREZA6', 'MES']:
    df_f[i] = df_f[i].astype(str)

# concatenando as colunas para o merge
df_x = df_f
df_x['concat'] = df_x['DESCRICAO_UO']
for i in ['PT', 'NATUREZA6']:
    df_x['concat'] = df_x['concat'] + df_x[i]

# base com as informacoes
info = pd.read_excel('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python/Base acompanhamento Layne.xlsx')

# fazendo o merge para cada valor e cada mês

for i in df_f['MES'].unique():
    info[i] = ''
    teste = df_x.loc[df_x['MES'] == i]
    for j in info['concat'].values:
     info.loc[info.concat == j, i] = teste.loc[teste.concat == j]['VALOR_EMPENHADO'].sum()

# Dropando o concat

info.drop(columns='concat', inplace=True)

# Padronizando as colunas e organiznado

info.rename(columns={'DESCRICAO_UO': 'UO', 'PT': 'PROGRAMA DE TRABALHO',
                      'PT_DESCRICAO': 'PROGRAMA DE TRABALHO DESCRIÇÃO',
                        'NATUREZA6': 'NATUREZA'}, inplace=True)

# Padronizando os meses

meses_padronizados = {
    '1': 'JAN', '2': 'FEV', '3': 'MAR', '4': 'ABR',
    '5': 'MAI', '6': 'JUN', '7': 'JUL', '8': 'AGO',
    '9': 'SET', '10': 'OUT', '11': 'NOV', '12': 'DEZ'
}

info.rename(columns=meses_padronizados, inplace=True)

# Padronizando os PTs

pt_padronizados = {'10302101550690000': '10.302. 1015. 5069',
                   '6122000420010000': '06.122. 0004. 2001',
                   '20605104335800000': '20.605. 1043. 3580',
                   '8306104352110000': '08.306. 1043. 5211',
                   '12368101237620000': '12.368. 1012. 3762',
                   '12368101237620000': '12.368. 1012. 3762',
                   '6181101738370000': '06.181. 1017. 3837',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14421000451850000': '14.421. 0004. 5185',
                   '14421000451850000': '14.421. 0004. 5185',
                   '14421000451850000': '14.421. 0004. 5185',
                   '6122101952750000': '06.122. 1019. 5275',
                   '6122101952750000': '06.122. 1019. 5275'}

info['PROGRAMA DE TRABALHO'] = info['PROGRAMA DE TRABALHO'].astype(str)
info['PROGRAMA DE TRABALHO'] = info['PROGRAMA DE TRABALHO'].map(pt_padronizados)


info.to_excel(base_arquivo)
info

,UO,PROGRAMA DE TRABALHO,NATUREZA,PROGRAMA DE TRABALHO DESCRIÇÃO,JAN,FEV,MAR,ABR
0,FUNDO ESTADUAL DE SAÚDE,10.302. 1015. 5069,333903600,QUALIFICAÇÃO DA ASSISTÊNCIA DE MÉDIA E ALTA CO...,62048990.06,63093518.99,60525791.43,64806749.2
1,POLÍCIA MILITAR DO ESTADO DE ALAGOAS,06.122. 0004. 2001,333904600,MANUTENÇÃO DAS ATIVIDADES DO ÓRGÃO,0.0,2478693.28,4905910.4,-109565.44
2,"SECRETARIA DE ESTADO DA AGRICULTURA, PECUÁRIA,...",20.605. 1043. 3580,333903200,FORTALECIMENTO DA AGRICULTURA FAMILIAR,0.0,2059515.64,2005239.07,12973536.67
3,SECRETARIA DE ESTADO DA ASSISTÊNCIA E DESENVOL...,08.306. 1043. 5211,333903900,APOIO A POPULAÇÃO EM SITUAÇÃO DE INSEGURANÇA A...,0.0,412648.1,392472.4,400520.52
4,SECRETARIA DE ESTADO DA EDUCAÇÃO,12.368. 1012. 3762,333901800,IMPLEMENTAÇÃO E FORTALECIMENTO DAS AÇÕES DO PR...,0.0,75000.0,949500.0,991700.0
5,SECRETARIA DE ESTADO DA EDUCAÇÃO,12.368. 1012. 3762,344901800,IMPLEMENTAÇÃO E FORTALECIMENTO DAS AÇÕES DO PR...,0.0,0.0,0.0,9800550.0
6,SECRETARIA DE ESTADO DA SEGURANÇA PÚBLICA,06.181. 1017. 3837,333904800,IMPLEMENTAÇÃO DO PROGRAMA FORÇA TAREFA,3496902.4,3747579.72,3595932.74,3739772.59
7,SECRETARIA DE ESTADO DE PREVENÇÃO À VIOLÊNCIA,14.122. 1017. 5189,333903600,MANUTENÇÃO DAS ATIVIDADES DO PROGRAMA RONDA NO...,1115083.62,1364896.39,1314256.32,1212863.91
8,SECRETARIA DE ESTADO DE PREVENÇÃO À VIOLÊNCIA,14.122. 1017. 5189,333903900,MANUTENÇÃO DAS ATIVIDADES DO PROGRAMA RONDA NO...,50000.0,198105.58,247216.56,238270.28
9,SECRETARIA DE ESTADO DE PREVENÇÃO À VIOLÊNCIA,14.122. 1017. 5189,333904700,MANUTENÇÃO DAS ATIVIDADES DO PROGRAMA RONDA NO...,0.0,242406.87,266722.12,271749.85
